In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

In [14]:
# 1. Loading and cleaning the Data
file_path = 'dataset.xlsx'
try:
    # List all sheets in excel file
    xl = pd.ExcelFile(file_path)
    print(f"Available sheets in the excel file: {xl.sheet_names}")
    sheets = pd.read_excel(file_path, sheet_name=None)

    sheet_names = list(sheets.keys())

    # extracting individual data names
    lecturer_details = sheets.get(sheet_names[0], pd.DataFrame())
    rooms_data = sheets.get(sheet_names[1], pd.DataFrame())
    courses_data = sheets.get(sheet_names[2], pd.DataFrame())
    student_requests = sheets.get(sheet_names[3], pd.DataFrame())

    print("Successfully loaded all data sheets")

    # Clean data - handling missing values

    lecturer_details = lecturer_details.fillna('')
    lecturer_details.columns = [col.strip().lower().replace(' ', '_') for col in lecturer_details.columns]

    rooms_data = rooms_data.fillna('')
    rooms_data.columns = [col.strip().lower().replace(' ', '_') for col in rooms_data.columns]

    courses_data = courses_data.fillna('')
    courses_data.columns = [col.strip().lower().replace(' ', '_') for col in courses_data.columns]

    student_requests = student_requests.fillna('')
    student_requests.columns = [col.strip().lower().replace(' ', '_') for col in student_requests.columns]

    print("\n--- Lecturer Details ---")
    print(f"Shape: {lecturer_details.shape}")
    display(lecturer_details.head())

    print("\n--- Rooms Data  ---")
    print(f"Shape: {rooms_data.shape}")
    display(rooms_data.head())

    print("\n--- Courses Data  ---")
    print(f"Shape: {courses_data.shape}")
    display(courses_data.head())

    print("\n--- Student requests ---")
    print(f"Shape: {student_requests.shape}")
    display(student_requests.head())


    data_dict = {
        'lecturer_details': lecturer_details.to_dict(orient='records'),
        'rooms_data': rooms_data.to_dict(orient='records'),
        'courses_data': courses_data.to_dict(orient='records'),
        'student_requests': student_requests.to_dict(orient='records')
    }


    # Save to JSON file
    with open('crestwood_data_cleaned.json', 'w') as f:
        json.dump(data_dict, f, indent=4)

    print("\n Data successfully converted to JSON and saved as 'crestwood_data_cleaned.json'")

    # Validations based on the rules
    print("\n--- Data validation ---")

    # Check if any course scheduled is unavailable blocks
    valid_blocks = ["1A", "1B", "2A", "2B", "3", "4A", "4B"]

    # Parse available and unavailable blocks
    def parse_blocks(block_str):
        if not block_str or block_str == '':
            return []
        return [b.strip() for b in str(block_str).split(',')]
    
    courses_data['available_blocks_list'] = courses_data['available_blocks'].apply(parse_blocks)
    courses_data['Unavailable_blocks_list'] = courses_data['unavailable_blocks'].apply(parse_blocks)

    # validate blocks
    invalid_blocks = []
    for _, course in courses_data.iterrows():
        for block in course['available_blocks_list']:
            if block not in valid_blocks:
                invalid_blocks.append((course['course_name'], block))

    if invalid_blocks:
        print(f"Found {len(invalid_blocks)} courses with invalid blocks:")
        for course, block in invalid_blocks:
            print(f"  - Course '{course}' has invalid block '{block}'")
    else:
        print("All courses have valid block assignments")

    # Insights
    print("\n--- Data Insights ---")

    # Count of courses by length/credits
    course_lengths = courses_data['course_length'].value_counts()
    print("\nCourse distribution by length:")
    print(course_lengths)

    # Count request priorities
    if 'priority' in student_requests.columns:
        priority_counts = student_requests['priority'].value_counts()
        print(priority_counts)

    # No of courses per lecturer
    lecturer_course_counts = lecturer_details['name'].value_counts()
    print("\nTop 5 lecturers by course load:")
    print(lecturer_course_counts.head())

    # Most requested courses
    if 'course' in student_requests.columns:
        popular_courses = student_requests['course'].value_counts()
        print("\nTop 5 most requested courses:")
        print(popular_courses.head())

    #Room capacity
    print("\nClassroom capacity stats:")
    print(rooms_data['capacity'].describe())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found")
except Exception as e:
    print(f"Error: {str(e)}")

Available sheets in the excel file: ['Lecturer Details', 'Rooms data', 'Course list', 'Student requests', 'RULES']
Successfully loaded all data sheets

--- Lecturer Details ---
Shape: (83, 6)


,lecturer_id,lecture_title,lecture_code,length,start_term,section_number
0,5361519,Band - High,ARTBND,2,1,1
1,5361487,Chorus - High,ARTChor,2,1,1
2,5361415,Drawing and Painting,ARTDRAW,2,1,1
3,5361415,Studio Art I,ARTSTD1,1,1,1
4,5361415,Studio Art II,ARTSTD2,1,2,1



--- Rooms Data  ---
Shape: (88, 10)


,course_title,section_number,year,term_description,prof_id,lecture_id,course_code,course_length,term_name,room_number
0,Chorus - High,1,2024 - 2025,1st Term,5361487,278027,ARTChor,4,1st,129
1,Drawing and Painting,1,2024 - 2025,1st Term,5361415,278028,ARTDRAW,4,1st,133
2,Studio Art I,1,2024 - 2025,1st Term,5361415,278032,ARTSTD1,2,1st,133
3,Bible 10,1,2024 - 2025,1st Term,5361400,278034,BIB10,4,1st,201
4,Bible 10,2,2024 - 2025,1st Term,5361400,278034,BIB10,4,1st,202



--- Courses Data  ---
Shape: (75, 11)


,course_code,title,length,priority,available_blocks,unavailable_blocks,minimum_section_size,target_section_size,maximum_section_size,number_of_sections,total_credits
0,ARTBND,Band - High,2.0,Core course,"1A, 1B, 2A, 2B, 3, 4A, 4B",,7,25,40,1,1.0
1,ARTChor,Chorus - High,2.0,Core course,"1A, 1B, 2A, 2B, 3, 4A, 4B",,7,20,40,1,1.0
2,ARTDRAW,Drawing and Painting,2.0,Core course,"1A, 1B, 2A, 2B, 3, 4A, 4B",,7,15,26,1,1.0
3,ARTSTD1,Studio Art I,1.0,Core course,"1A, 1B, 2A, 2B, 3, 4A, 4B",,7,15,26,1,0.5
4,ARTSTD2,Studio Art II,1.0,Core course,"1A, 1B, 2A, 2B, 3, 4A, 4B",,7,15,26,1,0.5



--- Student requests ---
Shape: (1259, 11)


,college_year,request_start_term,title,type,student_id,course_id,length,course_code,priority,department(s),credits
0,2nd Year,First term,Bible 10,Required,5407488,278034,2,BIB10,Core course,Bible,1.0
1,2nd Year,First term,English 10 Honors,Requested,5407488,278050,2,ENG10H,Core course,English,1.0
2,2nd Year,First term,Alg II Honors,Requested,5407488,278071,2,MATALG2H,Core course,Mathematics,1.0
3,2nd Year,First term,Physical Science,Requested,5407488,278099,2,SCIPHY,Core course,Science,1.0
4,2nd Year,First term,American Government & Politics,Requested,5407488,278100,2,SOC10GOV,Core course,Social Studies,1.0



 Data successfully converted to JSON and saved as 'crestwood_data_cleaned.json'

--- Data validation ---
All courses have valid block assignments

--- Data Insights ---
Error: 'course_length'
